In [115]:
import pandas as pd
import numpy as np
import datetime
import pandas_market_calendars as mcal
from pandas.tseries.offsets import BDay
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())

Data Loading and Cleaning

In [2]:
InflationRate = pd.read_csv('./data/1yrExpectedInflationRate.csv')
InflationRate.head(5)

In [18]:
CPI = pd.read_csv('./data/cpi.csv')
CPI.head()

,DATE,CPIAUCSL
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95


In [17]:
Inventory = pd.read_csv('./data/Inventory.csv',skiprows=4)
Inventory.head()

,Week of,Weekly U.S. Ending Stocks of Crude Oil and Petroleum Products Thousand Barrels
0,03/4/2022,1724594
1,02/25/2022,1735213
2,02/18/2022,1741514
3,02/11/2022,1745787
4,02/4/2022,1758364


In [20]:
USDIndex = pd.read_csv('./data/usd_index.csv')
USDIndex.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1971-01-04,120.529999,120.529999,120.529999,120.529999,120.529999,0.0
1,1971-01-05,120.519997,120.519997,120.519997,120.519997,120.519997,0.0
2,1971-01-06,120.489998,120.489998,120.489998,120.489998,120.489998,0.0
3,1971-01-07,120.550003,120.550003,120.550003,120.550003,120.550003,0.0
4,1971-01-08,120.529999,120.529999,120.529999,120.529999,120.529999,0.0


In [21]:
VIX = pd.read_csv('./data/VIX.csv')
VIX.head()

,DATE,VIXCLS
0,1990-01-02,17.24
1,1990-01-03,18.19
2,1990-01-04,19.22
3,1990-01-05,20.11
4,1990-01-08,20.26


In [22]:
wti = pd.read_excel('./data/PET_PRI_FUT_S1_D.xls', sheet_name=1,skiprows=2)
wti.head()

,Date,"Cushing, OK Crude Oil Future Contract 1 (Dollars per Barrel)","Cushing, OK Crude Oil Future Contract 2 (Dollars per Barrel)","Cushing, OK Crude Oil Future Contract 3 (Dollars per Barrel)","Cushing, OK Crude Oil Future Contract 4 (Dollars per Barrel)"
0,1983-03-30,NaN,NaN,29.35,NaN
1,1983-03-31,NaN,NaN,29.24,NaN
2,1983-04-04,29.44,NaN,29.10,NaN
3,1983-04-05,29.71,NaN,29.35,NaN
4,1983-04-06,29.92,NaN,29.50,NaN


In [24]:
pd.date_range('20200125',periods = 20,freq='M')

DatetimeIndex(['2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30',
               '2020-05-31', '2020-06-30', '2020-07-31', '2020-08-31',
               '2020-09-30', '2020-10-31', '2020-11-30', '2020-12-31',
               '2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30',
               '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31'],
              dtype='datetime64[ns]', freq='M')

In [106]:
def generate_exp_date(start_year, end_year):
    '''
    Generate expiry calendar for WTI futures according to the contract rule.
    '''
    exp_date = []
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start=str(start_year)+'-01-01', end=str(end_year)+'-12-31').to_pydatetime()
    for i in range(start_year, end_year+1):
        for j in range(1,13):
            _currdate = datetime.datetime(i,j,25)
            if _currdate in holidays or datetime.date.weekday(_currdate) in [5,6]:
                _currdate -= BDay(1)
            _currdate -= 3*US_BUSINESS_DAY
            exp_date.append(_currdate)
    return exp_date

In [173]:
def generate_exp_date(start_year, end_year):
    '''
    Generate expiry calendar for WTI futures according to the contract rule.
    '''
    exp_date = []
    tradingdays = mcal.get_calendar('CME_Agriculture').schedule(start_date='2005-01-01', end_date='2021-12-31').index.to_list()
    tradingdays.sort()
    for i in range(start_year, end_year+1):
        for j in range(1,13):
            _currdate = datetime.datetime(i,j,25)
            _currlist = [i for i in tradingdays if i <= _currdate]
            _currdate = _currlist[-4]
            exp_date.append(_currdate)
    return exp_date

In [174]:
exp_real = pd.read_excel('./data/Expiry Calendar HW1.xlsx')

In [175]:
exp_real['Test'] = [i for i in generate_exp_date(2005, 2021) if i >= pd.Timestamp('20051220')]

In [176]:
sum(exp_real['WTI'] == exp_real['Test'])

186

In [177]:
exp_real[exp_real['WTI'] != exp_real['Test']]

,WTI,RBOB,Test
11,2006-11-17,2006-11-30,2006-11-20
23,2007-11-16,2007-11-30,2007-11-19
24,2007-12-18,2007-12-31,2007-12-19
71,2011-11-18,2011-11-30,2011-11-21
83,2012-11-16,2012-11-30,2012-11-19
191,2021-11-22,2021-11-30,2021-11-19
192,2021-12-21,2021-12-31,2021-12-20


In [162]:
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=str(2005)+'-01-01', end=str(2021)+'-12-31').to_pydatetime()
datetime.datetime(2021,11,25) in holidays

True

In [141]:
[i for i in mcal.get_calendar_names() if 'NY' in i]

['NYMEX_Agriculture', 'NYFE', 'NYSE', 'XNYS']

In [156]:
[i for i in mcal.get_calendar_names() if 'CME' in i]

['CME_Equity',
 'CME_Agriculture',
 'CME_Rate',
 'CME_InterestRate',
 'CME_Bond',
 'CMES']